## Advanced CSS in Dash

#### A border on our app

```python
html.Div(dcc.Graph(figure=ecom_bar),
         style={'width':'500px',
                'height':'450px',
                'border':'5px dotted red'}
```

### CSS spacing
- To set the spacing of an HTML element
- Specify four numbers for each property(padding & margin)
- Clockwise will be top, right, bottom, left
- Alternatively: one number(will be applied to all sides)
- Alternatively: two numbers for top-bottom and left-right

### Centering with auto margin
- `margin : 100px auto`

### CSS for layout
- Elements not aligning? HTML elements can either be inline or block element
- **Inline render on the same line** : have no height or width (or box) properties
- Examples include <|strong|>, <|a|>, <|img|>
    
    
- **Block elements doesn't render side by side** : Block stack on top of one another as they include a line break
- Can't have more than one side-by-side
- Examples include <|h1|> , <|div|>


- There is an intermediatory option of **inline block**
- Can set height/width/box properties
- Can be side-by-side

### Example of Inline-block elements
- We can create some divs that are block and inline-block

```python
<div style='width:50px;height:50px;background-color:blue'></div>
<div style='width:50px;height:50px;background-color:red;display:inline-block'></div>
<div style='width:50px;height:50px;background-color:green;display:inline-block'></div>
```



```python
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
ecom_line = ecom_sales.groupby('Year-Month')['OrderValue'].agg('sum').reset_index(name='TotalSales')
line_fig = px.line(data_frame=ecom_line, x='Year-Month', y='TotalSales',title='Total Sales by Month')
line_fig.update_layout({'paper_bgcolor':'rgb(224, 255, 252)' }) 
ecom_bar = ecom_sales.groupby('Country')['OrderValue'].agg('sum').reset_index(name='TotalSales')
bar_fig = px.bar(data_frame=ecom_bar, x='TotalSales', y='Country', orientation='h',title='Total Sales by Country')
bar_fig.update_layout({'yaxis':{'dtick':1, 'categoryorder':'total ascending'}, 'paper_bgcolor':'rgb(224, 255, 252)'}) 

app = dash.Dash(__name__)

app.layout = html.Div(children=[
    html.Div(children=[
      html.Img(src=logo_link, 
               # Place the logo side-by-side the H1 with required margin
               style={'display':'inline-block', 'margin':'25px'}),
      html.H1(children=["Sales Figures"],
              # Make the H1 side-by-side with the logos
              style={'display':'inline-block'}), 
      html.Img(src=logo_link,
               # Place the logo side-by-side the H1 with required margin
               style={'display':'inline-block', 'margin':'25px'})]),
    html.Div(
        dcc.Graph(figure=line_fig), 
        # Ensure graphs are correct size, side-by-side with required margin
        style={'width':'500px', 'display':'inline-block', 'margin':'5px'}), 
    html.Div(
      	dcc.Graph(figure=bar_fig),
        # Ensure graphs are correct size, side-by-side with required margin
    	style={'width':'350px', 'display':'inline-block', 'margin':'5px' }), 
    html.H3(f"The largest order quantity was {ecom_sales.Quantity.max()}")
    ],style={'text-align':'center', 'font-size':22, 'background-color':'rgb(224, 255, 252)'})

if __name__ == '__main__':
    app.run_server(debug=True)
```

### Controlling object layout

```python
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
ecom_sales = pd.read_csv('/usr/local/share/datasets/ecom_sales.csv')
logo_link = 'https://assets.datacamp.com/production/repositories/5893/datasets/fdbe0accd2581a0c505dab4b29ebb66cf72a1803/e-comlogo.png'
ecom_bar_major_cat = ecom_sales.groupby('Major Category')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
ecom_bar_minor_cat = ecom_sales.groupby('Minor Category')['OrderValue'].agg('sum').reset_index(name='Total Sales ($)')
bar_fig_major_cat = px.bar(ecom_bar_major_cat, x='Total Sales ($)', y='Major Category', color='Major Category', color_discrete_map={'Clothes':'blue','Kitchen':'red','Garden':'green','Household':'yellow'})
bar_fig_minor_cat = px.bar(ecom_bar_minor_cat, x='Total Sales ($)', y='Minor Category')                       

app = dash.Dash(__name__)

app.layout = html.Div([
  html.Img(src=logo_link,
        # Add margin to the logo
        style={'margin':'30px 0px 0px 0px'}),
  html.H1("Sales breakdowns"),
  html.Div(children=[
      dcc.Graph(
        # Style the graphs to appear side-by-side
        figure=bar_fig_major_cat,
        style={'display':'inline-block'}),
      dcc.Graph(
        figure=bar_fig_minor_cat,
        style={'display':'inline-block'}),
  ]),
  html.H2("Major Category",
        # Style the titles to appear side-by-side with a 2 pixel border
        style={'display':'inline-block', 'border':'2px solid black',
        # Style the titles to have the correct spacings
               'padding':'10px', 'margin':'10px 220px'}),
  html.H2("Minor Category",
        # Style the titles to appear side-by-side with a 2 pixel border
        style={'display':'inline-block', 'border':'2px solid black',
        # Style the titles to have the correct spacings
               'padding':'10px', 'margin':'10px 220px'}),
  
  ], style={'text-align':'center', 'font-size':22})

if __name__ == '__main__':
    app.run_server(debug=True)
```

### Callbacks in Dash

#### What are callbacks?
- Functionality triggered by interaction
- A user interacts with an element -> A Python function is triggered --> something is changed
- This allows us to create interactive user expriences within our app


- Start with the `decorator` function 
- Uses `from dash.dependencies import Input, Output`
- Inside the decorator there are 2 components the Inputs & Outputs.
- **Output** defines which component will be changed using the return value of the triggered function. (where to send the function return)
- Specifically the `component_id` property identifies the component to update. And the `component_property` defines excatly what will be updated on that component.
- **Input** is what triggers the callback. `component_id` is similar & `component_property` is what will be taken from the triggering component to send to the triggered function, placed immediately below the decorator.

```python
@app.callback(
    Output(component_id='my_plot',
           component_property='figure'),
    Input(component_id='my_input',
          component_property='value')    
)
def some_function(data):
    # subset data
    # recreate figure
    return fig
```

### Dropdowns in Dash

```python
dcc.Dropdown(id='title_dd',
             options=[{'label':'Title 1',
                       'value':'Title 1'},
                      {'label':'Title 2',
                       'value':'Title 2'}])
```

- List of label-value dictionaries
- label is what the user sees and value is what is sent to the callbacks


### A dropdown callback
- Example to change the title
- Note that `id='title_dd'` is vital to link to the callback function
- We add the Graph we wish to change to the app layout

```python
app.layout = html.Div(children=[
    dcc.Dropdown(id='title_dd',
                 options=[{'label':'Title 1',
                           'value':'Title 1'},
                          {'label':'Title 2',
                           'value':'Title 2'}]),
    dcc.Graph(id='my_graph')])
@app.callback(
    Output(component_id='my_graph',
           component_property='figure'),
    Input(component_id='title_dd',
          component_property='value')
)

def update_plot(selection):
    title="None Selected"
    if selection:
        title = selection
    bar_fig = px.bar(
        data_frame=ecom_sales,
        title=f"{title}",
        x="Total Sales ($)", y='Country')
    return bar_fig
```



### Dropdown as a filter

```python
#@app.callback()
def update_plot(input_country):
    input_country = 'All Countries'
    sales = ecom_sales.copy(deep=True)
    if input_country:
        sales = sales[sales['Country'] == input_country]
    bar_fig = px.bar(
        data_frame=sales, title=f"Sales in {input_country}",
        x="Total Sales ($)", y='Country')
    return bar_fig
```